In [1]:
# blanket import modules
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
import pprint

out_csv = "../Output/happy_comb-data.csv"

In [2]:
# setting up paths for data
happy_cs = "../../Resources/World Happiness Report 2022.csv"
sunshine_cs = "../../Resources/sunshine hours by city.csv"
income_js = "../../Resources/Income.json"
alcohol_js = "../../Resources/Alcohol.json"

In [3]:
# making data pandas readable
happy_df = pd.read_csv(happy_cs)
sunshine_df = pd.read_csv(sunshine_cs)
income_df = pd.read_json(income_js)
alcohol_df = pd.read_json(alcohol_js)

In [4]:
#Happiness Data peek
happy_df.head(10)

,RANK,Country,Happiness score,Whisker-high,Whisker-low,Dystopia (1.83) + residual,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption
0,1,Finland,7.821,7.886,7.756,2.518,1.892,1.258,0.775,0.736,0.109,0.534
1,2,Denmark,7.636,7.710,7.563,2.226,1.953,1.243,0.777,0.719,0.188,0.532
2,3,Iceland,7.557,7.651,7.464,2.320,1.936,1.320,0.803,0.718,0.270,0.191
3,4,Switzerland,7.512,7.586,7.437,2.153,2.026,1.226,0.822,0.677,0.147,0.461
4,5,Netherlands,7.415,7.471,7.359,2.137,1.945,1.206,0.787,0.651,0.271,0.419
5,6,Luxembourg*,7.404,7.501,7.307,2.042,2.209,1.155,0.790,0.700,0.120,0.388
6,7,Sweden,7.384,7.454,7.315,2.003,1.920,1.204,0.803,0.724,0.218,0.512
7,8,Norway,7.365,7.440,7.290,1.925,1.997,1.239,0.786,0.728,0.217,0.474
8,9,Israel,7.364,7.426,7.301,2.634,1.826,1.221,0.818,0.568,0.155,0.143
9,10,New Zealand,7.200,7.279,7.120,1.954,1.852,1.235,0.752,0.680,0.245,0.483


In [5]:
#Sunshine Data peek
sunshine_df.head(2)

,Country,City,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
0,Afghanistan,Kabul,177.2,178.6,204.5,232.5,310.3,353.4,356.8,339.7,303.9,282.6,253.2,182.4,3175.1
1,Albania,Tirana,124.0,125.0,165.0,191.0,263.0,298.0,354.0,327.0,264.0,218.0,127.0,88.0,2544.0


In [6]:
#Income Data peek
income_df.head(2)

,country,medianIncome,meanIncome,gdpPerCapitaPPP,pop2022
0,Luxembourg,26321,31376,124590,642.371
1,United Arab Emirates,24292,27017,70089,10081.785


In [7]:
#Alcohol Data peek
alcohol_df.head(2)

,country,both,male,female
0,Latvia,13.19,21.61,6.31
1,Moldova,12.85,20.61,5.89


In [8]:
#calculating the avg sunshine for whole country then cutting just total
sun_ctry_df = sunshine_df.groupby("Country").mean().round(2)
sun_ctry_df.head(2)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Country,,,,,,,,,,,,,
Afghanistan,177.2,178.6,204.5,232.5,310.3,353.4,356.8,339.7,303.9,282.6,253.2,182.4,3175.1
Albania,124.0,125.0,165.0,191.0,263.0,298.0,354.0,327.0,264.0,218.0,127.0,88.0,2544.0


In [9]:
#grabbing data needed for merge
sun_yrly = sun_ctry_df.loc[: , "Year"]
mean_inc = income_df.loc[: , ["country" , "meanIncome"]]
alc_cons = alcohol_df.loc[: , ["country" , "both"]]
happ_small = happy_df.loc[: , ["RANK", "Country", "Happiness score", "Explained by: Healthy life expectancy"]]

In [10]:
coun_merge = pd.merge(mean_inc,alc_cons, on="country")
coun_2_Coun = coun_merge.rename(columns={"country":"Country"})
sub_comb_data =  pd.merge(pd.merge(happ_small,sun_yrly, on="Country", how="left")
                        ,coun_2_Coun, on="Country", how="left").dropna()
sub_comb_data = sub_comb_data.rename(columns={"Happiness score" : "Happiness Score",
                                              "Year":"Sunshine Hrs per Year", "meanIncome":"Average Income ($)", 
                                              "both":"Alcohol Consumption (L/year)", 
                                              "Explained by: Healthy life expectancy" : "Healthy Life Expectancy"})
sub_comb_data

,RANK,Country,Happiness Score,Healthy Life Expectancy,Sunshine Hrs per Year,Average Income ($),Alcohol Consumption (L/year)
0,1,Finland,7.821,0.775,1858.00,18742.0,10.65
1,2,Denmark,7.636,0.777,1932.00,20304.0,10.13
2,3,Iceland,7.557,0.803,1326.00,19300.0,9.21
3,4,Switzerland,7.512,0.822,1932.75,25787.0,11.23
4,5,Netherlands,7.415,0.787,1662.00,19690.0,9.67
...,...,...,...,...,...,...,...
135,136,India,3.777,0.471,2517.02,1314.0,5.61
136,137,Zambia,3.760,0.306,2965.47,1045.0,4.46
137,138,Malawi,3.750,0.388,2795.87,733.0,4.08
138,139,Tanzania,3.702,0.425,3032.85,992.0,12.04


In [11]:
#output data to csv
sub_comb_data.loc[(sub_comb_data.Country == 'United States'),'Country']='United States of America'
happy_data = sub_comb_data.to_csv(out_csv, index=False)